In [1]:
%load_ext autoreload
%autoreload 2

Download if file is not in folder.

In [2]:
from litedict import SQLDict

In [3]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

Basic functionality

In [4]:
d = SQLDict(":memory:")

d[TEST_1] = "asdfoobar"

assert d[TEST_1] == "asdfoobar"

del d[TEST_1]

assert d.get(TEST_1, None) is None

Glob matching

In [5]:
d[TEST_1] = "asdfoobar"

d[TEST_2] = "foobarasd"

d["key_testx_3"] = "barasdfoo"

assert d.glob("key_test*") == ["asdfoobar", "foobarasd", "barasdfoo"]

assert d.glob("key_test_?") == ["asdfoobar", "foobarasd"]

assert d.glob("key_tes[tx]*") == ["asdfoobar", "foobarasd", "barasdfoo"]

Numbers

In [6]:
d[TEST_1] = 1

d[TEST_2] = 2

assert d[TEST_1] + d[TEST_2] == 3

In [7]:
with d.transaction():
    d["asd"] = "efg"
    d["foo"] = "bar"
    assert d.conn.in_transaction

In [8]:
try:
    with d.transaction():
        d["failed"] = "no"

        assert d.conn.in_transaction

        raise Exception
except:
    # check the transaction succesfully rolled back
    assert d.get("failed") is None

## Benchmarks

In [9]:
from string import ascii_lowercase, printable
from random import choice
import random


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [10]:
import gc

import pickle

import json

**Pickle**

In [11]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [12]:
gc.collect()

110

In [13]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

82.2 µs ± 6.66 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**Noop**

In [14]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: x,
    decoder=lambda x: x,
)

In [15]:
gc.collect()

0

In [16]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

82.4 µs ± 4.83 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**JSON**

In [17]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: json.dumps(x),
    decoder=lambda x: json.loads(x),
)

In [18]:
gc.collect()

69

In [19]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

95.4 µs ± 24.5 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**Pickle Python obj**

In [20]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [21]:
gc.collect()

69

In [22]:
class C:
    def __init__(self, x):
        self.x = x

    def pp(self):
        return x

    def f(self):
        def _f(y):
            return y * self.x ** 2

        return _f

In [23]:
%%timeit -n20000 -r10

d[random_string(8)] = C(random.randint(1, 200))

d.get(random_string(8), None)

58.6 µs ± 21.2 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)


**Dictionary**

In [24]:
d = {}

In [25]:
gc.collect()

91

In [26]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

59.4 µs ± 2.59 µs per loop (mean ± std. dev. of 10 runs, 20000 loops each)
